In [2]:
import math
import random
import copy
import numpy as np
import pandas as pd
import time

# =============================================================================
# --- DATA LOADING ---
# =============================================================================
def charger_donnees_depuis_excel(chemin_fichier):
    print(f"--- Chargement des données depuis '{chemin_fichier}' ---")
    donnees = {}
    CONVERSION_MILE_TO_M = 1609.34
    df_puits = pd.read_excel(chemin_fichier, sheet_name='Puits')
    puits_data = {}
    pipe_costs_par_m = {4: round(358.69 / CONVERSION_MILE_TO_M, 5), 6: round(454.06 / CONVERSION_MILE_TO_M, 5)}
    for index, row in df_puits.iterrows():
        nom_puits, diam, debit_str = row['NomPuits'], row['DiametreFlowline_in'], row['Debit_kSm3_day']
        if diam not in pipe_costs_par_m: raise ValueError(f"Coût non défini pour le diamètre {diam}in.")
        puits_data[nom_puits] = {'x': float(row['X']), 'y': float(row['Y']), 'production_sm3d': float(str(debit_str).replace(',', '.')) * 1000, 'diametre_flowline_in': diam, 'CDF_i_dollar_par_m': pipe_costs_par_m[diam]}
    donnees['puits'] = puits_data
    print(f"Données des puits chargées: {len(puits_data)} puits.")
    df_manifolds = pd.read_excel(chemin_fichier, sheet_name='TypesManifold')
    donnees['manifolds'] = {row['TypeID']: {'capacite_puits': int(row['CapacitePuits']), 'cout_installation_dollar': float(row['CoutInstallation_dollar'])} for index, row in df_manifolds.iterrows()}
    print(f"Types de manifolds chargés: {len(donnees['manifolds'])} types.")
    df_trunklines = pd.read_excel(chemin_fichier, sheet_name='SpecsTrunkline')
    donnees['trunklines'] = sorted([{'diametre_in': int(r['Diametre_in']), 'cout_par_m_dollar': round(float(r['CoutParMile_dollar']) / CONVERSION_MILE_TO_M, 5), 'min_total_prod_sm3d': int(r['MinProd_Sm3d']), 'max_total_prod_sm3d': int(r['MaxProd_Sm3d'])} for i, r in df_trunklines.iterrows()], key=lambda x: x['min_total_prod_sm3d'])
    print(f"Spécifications de trunklines chargées: {len(donnees['trunklines'])} specs.")
    return donnees

# =============================================================================
# --- UTILITY AND COST FUNCTIONS ---
# =============================================================================
def distance(p1, p2): return math.sqrt((p1.get('x', 0) - p2.get('x', 0))**2 + (p1.get('y', 0) - p2.get('y', 0))**2)

def get_trunkline_params(prod, specs):
    if not specs: return None, None
    for spec in specs:
        if spec['min_total_prod_sm3d'] <= prod <= spec['max_total_prod_sm3d']: return spec['diametre_in'], spec['cout_par_m_dollar']
    if prod > 0:
        s_specs = sorted(specs, key=lambda x: x['max_total_prod_sm3d'])
        if not s_specs: return None, None
        if prod > s_specs[-1]['max_total_prod_sm3d']: return s_specs[-1]['diametre_in'], s_specs[-1]['cout_par_m_dollar']
        if prod < s_specs[0]['min_total_prod_sm3d']: return s_specs[0]['diametre_in'], s_specs[0]['cout_par_m_dollar']
    return None, None

def get_zone_etude(puits):
    if not puits: return 0, 0, 0, 0
    all_x, all_y = [d['x'] for d in puits.values()], [d['y'] for d in puits.values()]
    return min(all_x), max(all_x), min(all_y), max(all_y)

def generer_sites_candidats_grille(x_min, x_max, y_min, y_max, pas, marge):
    sites = []
    for x in np.arange(x_min - marge, x_max + marge + pas / 2, pas):
        for y in np.arange(y_min - marge, y_max + marge + pas / 2, pas):
            sites.append({'x': round(x, 2), 'y': round(y, 2)})
    return sites

def calculer_cout_total_complet(sol, puits, m_types, t_specs, cpf_cost):
    if not sol or not sol.get('manifolds_ouverts') or not sol.get('cpf_location'): return float('inf')
    cost = cpf_cost
    active_manifolds = {k: v for k, v in sol['manifolds_ouverts'].items() if v.get('puits_connectes')}
    for info in active_manifolds.values(): cost += m_types[info['type_id']]['cout_installation_dollar']
    for p_id, m_id in sol['affectations_puits'].items():
        if m_id not in active_manifolds: return float('inf')
        cost += distance(puits[p_id], active_manifolds[m_id]) * puits[p_id]['CDF_i_dollar_par_m']
    cpf_loc = sol['cpf_location']
    for m_id, info in active_manifolds.items():
        prod = sum(puits[p_id]['production_sm3d'] for p_id in info['puits_connectes'])
        d, c = get_trunkline_params(prod, t_specs)
        sol['manifolds_ouverts'][m_id]['total_production_sm3d'] = prod
        sol['manifolds_ouverts'][m_id]['DTj_in'] = d
        sol['manifolds_ouverts'][m_id]['CDTj_dollar_par_m'] = c
        if c: cost += distance(info, cpf_loc) * c
        elif prod > 0: return float('inf')
    return cost

# =============================================================================
# --- ORIGINAL CONSTRUCTION HEURISTIC ---
# =============================================================================
def construire_solution_clustering(puits_data, types_manifold_data):
    """
    This is your original heuristic. It places manifolds at the exact centroid.
    The only modification is using 'MFD-n' for IDs for better readability.
    """
    clusters = {p_id: {'puits': {p_id}, 'center': p_data} for p_id, p_data in puits_data.items()}
    max_cap = max(info['capacite_puits'] for info in types_manifold_data.values())
    while True:
        if len(clusters) <= 1: break
        meilleure_fusion = {'dist': float('inf'), 'pair': None}
        cluster_ids = list(clusters.keys())
        for i in range(len(cluster_ids)):
            for j in range(i + 1, len(cluster_ids)):
                id1, id2 = cluster_ids[i], cluster_ids[j]
                if len(clusters[id1]['puits']) + len(clusters[id2]['puits']) <= max_cap:
                    dist = distance(clusters[id1]['center'], clusters[id2]['center'])
                    if dist < meilleure_fusion['dist']:
                        meilleure_fusion = {'dist': dist, 'pair': (id1, id2)}
        if meilleure_fusion['pair'] is None: break
        id1, id2 = meilleure_fusion['pair']
        puits_fusionnes = clusters[id1]['puits'].union(clusters[id2]['puits'])
        centre_x = sum(puits_data[p]['x'] for p in puits_fusionnes) / len(puits_fusionnes)
        centre_y = sum(puits_data[p]['y'] for p in puits_fusionnes) / len(puits_fusionnes)
        nouveau_id = f"{id1}-{id2}"
        del clusters[id1], clusters[id2]
        clusters[nouveau_id] = {'puits': puits_fusionnes, 'center': {'x': centre_x, 'y': centre_y}}

    solution = {'manifolds_ouverts': {}, 'affectations_puits': {}}
    types_tries = sorted(types_manifold_data.items(), key=lambda item: item[1]['capacite_puits'])
    for i, cluster in enumerate(clusters.values()):
        manifold_id = f"MFD-{i+1}"
        taille_cluster = len(cluster['puits'])
        type_choisi_id = next((tid for tid, tinfo in types_tries if taille_cluster <= tinfo['capacite_puits']), None)
        if type_choisi_id is None: continue
        
        solution['manifolds_ouverts'][manifold_id] = {
            'type_id': type_choisi_id,
            'x': cluster['center']['x'], 
            'y': cluster['center']['y'],
            'puits_connectes': cluster['puits']
        }
        for p_id in cluster['puits']:
            solution['affectations_puits'][p_id] = manifold_id
    return solution

def optimiser_cpf_pour_manifolds_fixes(solution_p1, puits_data, types_manifold_data, sites_candidats_cpf, trunkline_specs_data, cpf_cost):
    if not solution_p1 or not solution_p1.get('manifolds_ouverts'): return None, float('inf')
    meilleure_solution, meilleur_cout = None, float('inf')
    active_manifolds = {k: v for k, v in solution_p1['manifolds_ouverts'].items() if v.get('puits_connectes')}
    if not active_manifolds: return None, float('inf')
    solution_base = copy.deepcopy(solution_p1)
    solution_base['manifolds_ouverts'] = active_manifolds
    
    for cpf_coords in sites_candidats_cpf:
        solution_temp = copy.deepcopy(solution_base)
        solution_temp['cpf_location'] = cpf_coords
        cout_actuel = calculer_cout_total_complet(solution_temp, puits_data, types_manifold_data, trunkline_specs_data, cpf_cost)
        if cout_actuel < meilleur_cout:
            meilleur_cout, meilleure_solution = cout_actuel, copy.deepcopy(solution_temp)
            meilleure_solution['cout_total'] = meilleur_cout
            
    return meilleure_solution, meilleur_cout

# =============================================================================
# --- SIMULATED ANNEALING ---
# =============================================================================
def generer_voisin(solution, puits_data, types_manifold_data, sites_candidats, prob_moves):
    voisin = copy.deepcopy(solution)
    move_type = random.choices(list(prob_moves.keys()), weights=list(prob_moves.values()))[0]

    active_manifold_ids = [m_id for m_id, m_data in voisin['manifolds_ouverts'].items() if m_data.get('puits_connectes')]
    if not active_manifold_ids: return voisin

    if move_type == 'reassign_well' and len(active_manifold_ids) > 1:
        p_id = random.choice(list(voisin['affectations_puits'].keys()))
        m_id_origine = voisin['affectations_puits'].get(p_id)
        if not m_id_origine: return voisin
        
        candidats_dest = [m_id for m_id in active_manifold_ids if m_id != m_id_origine and len(voisin['manifolds_ouverts'][m_id]['puits_connectes']) < types_manifold_data[voisin['manifolds_ouverts'][m_id]['type_id']]['capacite_puits']]
        if not candidats_dest: return voisin
        m_id_dest = random.choice(candidats_dest)
        
        voisin['affectations_puits'][p_id] = m_id_dest
        voisin['manifolds_ouverts'][m_id_origine]['puits_connectes'].remove(p_id)
        voisin['manifolds_ouverts'][m_id_dest]['puits_connectes'].add(p_id)
        
    elif move_type == 'relocate_manifold':
        m_id = random.choice(active_manifold_ids)
        new_site = random.choice(sites_candidats)
        voisin['manifolds_ouverts'][m_id]['x'] = new_site['x']
        voisin['manifolds_ouverts'][m_id]['y'] = new_site['y']

    elif move_type == 'relocate_cpf':
        new_site = random.choice(sites_candidats)
        voisin['cpf_location'] = new_site
        
    return voisin

def recuit_simule(solution_initiale, puits_data, types_manifold_data, trunkline_specs_data, sites_candidats, cpf_cost, params, lock_cpf=False):
    titre_sa = "AVEC CPF VERROUILLÉ" if lock_cpf else "AVEC CPF OPTIMISÉ"
    print(f"\n--- DÉBUT DU RECUIT SIMULÉ ({titre_sa}) ---")
    start_time = time.time()
    
    T, T_FINAL, ALPHA, N_ITER_TEMP = params['T_INITIAL'], params['T_FINAL'], params['ALPHA'], params['N_ITER_TEMP']
    
    prob_moves = copy.deepcopy(params['PROB_MOVES'])
    if lock_cpf:
        cpf_prob = prob_moves.pop('relocate_cpf', 0)
        if prob_moves:
            total_prob = sum(prob_moves.values())
            if total_prob > 0:
                for key in prob_moves:
                    prob_moves[key] += cpf_prob * (prob_moves[key] / total_prob)

    sol_actuelle = copy.deepcopy(solution_initiale)
    cout_actuel = calculer_cout_total_complet(sol_actuelle, puits_data, types_manifold_data, trunkline_specs_data, cpf_cost)
    
    meilleure_sol, meilleur_cout = copy.deepcopy(sol_actuelle), cout_actuel
    
    while T > T_FINAL:
        for _ in range(N_ITER_TEMP):
            sol_voisine = generer_voisin(sol_actuelle, puits_data, types_manifold_data, sites_candidats, prob_moves)
            cout_voisin = calculer_cout_total_complet(sol_voisine, puits_data, types_manifold_data, trunkline_specs_data, cpf_cost)
            delta_cout = cout_voisin - cout_actuel
            if delta_cout < 0 or (T > 0 and random.random() < math.exp(-delta_cout / T)):
                sol_actuelle, cout_actuel = sol_voisine, cout_voisin
                if cout_actuel < meilleur_cout:
                    meilleur_cout = cout_actuel
                    meilleure_sol = copy.deepcopy(sol_actuelle)
        T *= ALPHA
        print(f"Temp: {T:.2f}, Coût Actuel: ${cout_actuel:,.2f}, Meilleur Coût: ${meilleur_cout:,.2f}", end="\r")

    end_time = time.time()
    print(f"\n--- FIN DU RECUIT SIMULÉ (Durée: {end_time - start_time:.2f} secondes) ---")
    
    meilleure_sol['cout_total'] = meilleur_cout
    return meilleure_sol, meilleur_cout

# =============================================================================
# --- AFFICHAGE & SCÉNARIOS ---
# =============================================================================
def afficher_resultats_detailles(solution_finale, types_manifold_data, titre=""):
    print(f"\n\n--- RÉSULTAT FINAL DÉTAILLÉ ({titre}) ---")
    if not solution_finale or not solution_finale.get('manifolds_ouverts'):
        print("Aucune solution valide à afficher.")
        return
        
    cout_total = solution_finale.get('cout_total', 0)
    cpf_loc = solution_finale.get('cpf_location', {'x': 'N/A', 'y': 'N/A'})
    print(f"Coût Total Estimé: ${cout_total:,.2f}")
    print(f"Localisation CPF: ({cpf_loc.get('x', 'N/A'):.2f}, {cpf_loc.get('y', 'N/A'):.2f})")
    
    manifolds_actifs = {k: v for k, v in solution_finale.get('manifolds_ouverts', {}).items() if v.get('puits_connectes')}
    print(f"\nNombre de Manifolds Actifs: {len(manifolds_actifs)}")
    print("-" * 50)
    
    for m_id, m_data in sorted(manifolds_actifs.items()):
        type_id, capacite = m_data.get('type_id', 'N/A'), types_manifold_data.get(m_data.get('type_id'), {}).get('capacite_puits', 'N/A')
        print(f"  Manifold ID: {m_id}")
        print(f"    - Coordonnées: ({m_data.get('x', 0):.2f}, {m_data.get('y', 0):.2f})")
        print(f"    - Type: {type_id} (Capacité: {capacite})")
        print(f"    - Trunkline vers CPF: Diamètre {m_data.get('DTj_in', 'N/A')} in")
        puits_connectes = sorted(list(m_data.get('puits_connectes', [])))
        print(f"    - Puits Connectés ({len(puits_connectes)}):")
        if puits_connectes:
            for i in range(0, len(puits_connectes), 6): print(f"      {'  '.join(puits_connectes[i:i+6])}")
        else: print("      Aucun")
        print("-" * 50)

def lancer_scenario_cpf_fixe(donnees, params, sites_candidats):
    print("\n" + "="*70 + "\n--- DÉMARRAGE SCÉNARIO 1 : CPF AVEC EMPLACEMENT FIXE ---\n" + "="*70)
    PUITS_DATA = donnees['puits']
    TYPES_MANIFOLD_DATA = donnees['manifolds']
    TRUNKLINE_SPECS_DATA = donnees['trunklines']

    sol_initiale = construire_solution_clustering(PUITS_DATA, TYPES_MANIFOLD_DATA)
    sol_initiale['cpf_location'] = {'x': params['CPF_FIXE_X'], 'y': params['CPF_FIXE_Y']}
    cout_initial = calculer_cout_total_complet(sol_initiale, PUITS_DATA, TYPES_MANIFOLD_DATA, TRUNKLINE_SPECS_DATA, params['CPF_INSTALLATION_COST_DOLLAR_VAL'])
    sol_initiale['cout_total'] = cout_initial
    afficher_resultats_detailles(sol_initiale, TYPES_MANIFOLD_DATA, "Initial pour CPF Fixe")

    sol_finale, cout_final = recuit_simule(sol_initiale, PUITS_DATA, TYPES_MANIFOLD_DATA, TRUNKLINE_SPECS_DATA, sites_candidats, params['CPF_INSTALLATION_COST_DOLLAR_VAL'], params, lock_cpf=True)
    afficher_resultats_detailles(sol_finale, TYPES_MANIFOLD_DATA, "Amélioré pour CPF Fixe")
    return cout_initial, cout_final

def lancer_scenario_cpf_optimise(donnees, params, sites_candidats):
    print("\n" + "="*70 + "\n--- DÉMARRAGE SCÉNARIO 2 : CPF AVEC EMPLACEMENT OPTIMISÉ ---\n" + "="*70)
    PUITS_DATA = donnees['puits']
    TYPES_MANIFOLD_DATA = donnees['manifolds']
    TRUNKLINE_SPECS_DATA = donnees['trunklines']

    sol_p1 = construire_solution_clustering(PUITS_DATA, TYPES_MANIFOLD_DATA)
    sol_initiale, cout_initial = optimiser_cpf_pour_manifolds_fixes(sol_p1, PUITS_DATA, TYPES_MANIFOLD_DATA, sites_candidats, TRUNKLINE_SPECS_DATA, params['CPF_INSTALLATION_COST_DOLLAR_VAL'])
    afficher_resultats_detailles(sol_initiale, TYPES_MANIFOLD_DATA, "Initial pour CPF Optimisé")

    sol_finale, cout_final = recuit_simule(sol_initiale, PUITS_DATA, TYPES_MANIFOLD_DATA, TRUNKLINE_SPECS_DATA, sites_candidats, params['CPF_INSTALLATION_COST_DOLLAR_VAL'], params, lock_cpf=False)
    afficher_resultats_detailles(sol_finale, TYPES_MANIFOLD_DATA, "Amélioré pour CPF Optimisé")
    return cout_initial, cout_final

# =============================================================================
# --- MAIN EXECUTION BLOCK ---
# =============================================================================
if __name__ == "__main__":
    try:
        fichier_excel = r'C:\Users\Dell\Downloads\donnees_probleme.xlsx'
        donnees = charger_donnees_depuis_excel(fichier_excel)
        
        PARAMS = {
            'CPF_INSTALLATION_COST_DOLLAR_VAL': 450000000.00,
            'MARGE_POUR_GRILLE_M_VAL': 5000.0,
            'PAS_GRILLE_MANIFOLD_M_VAL': 2000.0,
            'T_INITIAL': 100000, 'T_FINAL': 1, 'ALPHA': 0.99, 'N_ITER_TEMP': 100,
            'PROB_MOVES': { 'reassign_well': 0.5, 'relocate_manifold': 0.3, 'relocate_cpf': 0.2 },
            'CPF_FIXE_X': 449397.35, 'CPF_FIXE_Y': 3048477.39,
        }
        
        PUITS_DATA = donnees['puits']
        xmin, xmax, ymin, ymax = get_zone_etude(PUITS_DATA)
        SITES_CANDIDATS = generer_sites_candidats_grille(xmin, xmax, ymin, ymax, PARAMS['PAS_GRILLE_MANIFOLD_M_VAL'], PARAMS['MARGE_POUR_GRILLE_M_VAL'])
        print(f"\nNombre de sites candidats générés: {len(SITES_CANDIDATS)}")

        cout_initial_fixe, cout_final_fixe = lancer_scenario_cpf_fixe(donnees, PARAMS, SITES_CANDIDATS)
        cout_initial_opt, cout_final_opt = lancer_scenario_cpf_optimise(donnees, PARAMS, SITES_CANDIDATS)
        
        print("\n\n" + "="*70 + "\n--- RÉSUMÉ DES COMPARAISONS ---\n" + "="*70)
        print(f"SCÉNARIO 1 (CPF FIXE):")
        print(f"  - Coût initial:     ${cout_initial_fixe:,.2f}")
        print(f"  - Coût final (SA):  ${cout_final_fixe:,.2f}")
        if cout_initial_fixe > 0:
            print(f"  - Amélioration:     {((cout_initial_fixe - cout_final_fixe) / cout_initial_fixe) * 100:.2f}%")
        
        print("\nSCÉNARIO 2 (CPF OPTIMISÉ):")
        print(f"  - Coût initial:     ${cout_initial_opt:,.2f}")
        print(f"  - Coût final (SA):  ${cout_final_opt:,.2f}")
        if cout_initial_opt > 0:
            print(f"  - Amélioration:     {((cout_initial_opt - cout_final_opt) / cout_initial_opt) * 100:.2f}%")

    except FileNotFoundError:
        print(f"\nERREUR CRITIQUE: Le fichier '{fichier_excel}' n'a pas été trouvé.")
    except Exception as e:
        import traceback
        traceback.print_exc()
        print(f"\nUNE ERREUR EST SURVENUE: {e}")

--- Chargement des données depuis 'C:\Users\Dell\Downloads\donnees_probleme.xlsx' ---
Données des puits chargées: 143 puits.
Types de manifolds chargés: 3 types.
Spécifications de trunklines chargées: 13 specs.

Nombre de sites candidats générés: 5390

--- DÉMARRAGE SCÉNARIO 1 : CPF AVEC EMPLACEMENT FIXE ---


--- RÉSULTAT FINAL DÉTAILLÉ (Initial pour CPF Fixe) ---
Coût Total Estimé: $531,357,652.83
Localisation CPF: (449397.35, 3048477.39)

Nombre de Manifolds Actifs: 11
--------------------------------------------------
  Manifold ID: MFD-1
    - Coordonnées: (473560.81, 2935659.68)
    - Type: MFD_Cap15 (Capacité: 15)
    - Trunkline vers CPF: Diamètre 10 in
    - Puits Connectés (14):
      chmp2-10  chmp2-11  chmp2-12  chmp2-13  chmp2-14  chmp2-15
      chmp2-3  chmp2-4  chmp2-W11  chmp2-W4  chmp2-W6  chmp2-W7
      chmp2-W8  chmp2-W9
--------------------------------------------------
  Manifold ID: MFD-10
    - Coordonnées: (483485.21, 2917091.55)
    - Type: MFD_Cap15 (Capacité: